In [ ]:
import torch
import torchvision
from torchvision import transforms
import os
import imageio
import numpy as np
import pandas as pd
from statistics import mean  
from math import ceil  
from matplotlib import pyplot as plt
import cv2 as cv
from google.colab import files
from skimage.transform import rotate, AffineTransform, warp
import random
from skimage.util import random_noise
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip '/content/drive/Shared drives/HaiHua Garbage Classification /Justin/Train_Resized_224.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Train_Resized_224/76380.png  
  inflating: Train_Resized_224/76382.png  
 extracting: Train_Resized_224/76384.png  
 extracting: Train_Resized_224/76385.png  
  inflating: Train_Resized_224/76387.png  
 extracting: Train_Resized_224/76388.png  
 extracting: Train_Resized_224/76389.png  
 extracting: Train_Resized_224/76392.png  
 extracting: Train_Resized_224/76394.png  
 extracting: Train_Resized_224/76395.png  
 extracting: Train_Resized_224/76396.png  
  inflating: Train_Resized_224/76397.png  
 extracting: Train_Resized_224/76398.png  
 extracting: Train_Resized_224/76401.png  
 extracting: Train_Resized_224/76402.png  
 extracting: Train_Resized_224/76403.png  
 extracting: Train_Resized_224/76404.png  
 extracting: Train_Resized_224/76407.png  
 extracting: Train_Resized_224/76408.png  
 extracting: Train_Resized_224/90990.png  
 extracting: Train_Resized_224/90991.png  
 extracting: Train_Resized_224/90992.png  
 ex

### This dataloader returns a y_in containing [[0,1]] OR [[1,0]] (dim=1 compatible) instead of the usual 204 classes. 

**Modify the targeting_class variable according to your needs! **

In [ ]:
def dataloader(batchsize): 
  labels = pd.read_csv(
      '/content/drive/Shared drives/HaiHua Garbage Classification /Train_data_320.csv')  

  targeting_class = 28 
  #                 ^ modify the targeting_class according to your needs! 


  idx = labels['image_id']  
  labels = labels.set_index('image_id') 
  np.random.shuffle(idx)  

  x = [] 
  y = []  
  #random = 10
  for i, image_name in enumerate(idx):  
      if i % batchsize == 0 and i != 0:  
          x_in = torch.tensor(x, dtype=torch.float32).cuda()  
          y_in = torch.tensor(y, dtype=torch.float32).cuda()  
          x = [] 
          y = [] 
          yield (x_in, y_in) 

      image_path = '/content/Train_Resized_224/%s.png' % image_name  
      image = cv.imread(image_path) 
      number = random.randint(0,10)
      if number <= 5:
        noisy_image = image + np.random.rand(224, 224, 3)*30
        noisy_image = np.divide(noisy_image, 255)
        noisy_image = np.subtract(noisy_image, np.array([0.485, 0.456, 0.406]))
        noisy_image = np.divide(noisy_image, np.array([0.229, 0.224, 0.225]))
        image = np.reshape(noisy_image, (3,224,224))

      else: 
        image = np.divide(image, 255)
        image = np.subtract(image, np.array([0.485, 0.456, 0.406]))
        image = np.divide(image, np.array([0.229, 0.224, 0.225]))
        image = np.reshape(image, (3, 224, 224))

      
      id = labels.at[int(image_name), 'id']  
      x.append(image)  
      
      y0 = np.zeros(2)  
      if int(id) != int(targeting_class): 
        y0[0]=1
      else:
        y0[1]=1

      y.append(y0)


In [ ]:
def get_accuracy(prediction, target):  

    pred = torch.argmax(prediction, dim=1) 
    true = torch.argmax(target, dim=1) 
    n_correct = torch.sum(torch.eq(pred, true)) 
    n_total = len(pred) 
    accuracy = n_correct / n_total
    
    return accuracy.item()

### DO NOT LOAD 204-class .pt models, cuz they are incompatible!! Only load previously trained binary-class models! 

In [ ]:
BATCHSIZE = 256 
EPOCH = 32
best_acc = 0
torch.backends.cudnn.enabled = False
n_train = ceil(70000 / BATCHSIZE) 
n_batches = ceil(80000 / BATCHSIZE)

print('loading model...')
model = torchvision.models.resnext50_32x4d(pretrained=True)


for i, child in enumerate(model.children()): 
    if i < 7:
        for param in child.parameters():
            param.requires_grad = False

model.fc = torch.nn.Linear(2048, 256, bias=True)

model = torch.nn.Sequential(model, torch.nn.Linear(256,2, bias=True), torch.nn.Softmax(dim=1)).to('cuda')


model = torch.load(f'/content/drive/Shared drives/HaiHua Garbage Classification /Justin pretrained networks/One-VS-All/Apr12_ONE_VS_ALL_LABEL28_WITH_IA.pt')
# ^ DO NOT LOAD MODELS UNLESS THEY ARE TRAINED BINARY, ELSE ERRORS WOULD OCCUR


print('training...')
print('trains for  %s steps per epoch' % n_train)
optimizer = torch.optim.Adam(lr=0.0001, params=list(model.parameters()))
criterion = torch.nn.BCELoss()

for epoch in range(EPOCH):
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    tbar = tqdm(total=n_batches)
    for i, (x, y) in enumerate(dataloader(BATCHSIZE)):
        if i < n_train:
            model.train()
            model.zero_grad()
            output = model(x) 
            loss = criterion(output, y)
            acc = get_accuracy(output, y)
            loss.backward(retain_graph=False)
            optimizer.step()
            train_loss.append(loss.item())
            train_acc.append(acc)
            tbar.set_description('loss %s' % round(mean(train_loss), 10))
            tbar.update()            

        else:
            with torch.no_grad():
                model.eval()

                model.zero_grad()
                output = model(x)
                loss = criterion(output, y)
                acc = get_accuracy(output, y)

                val_loss.append(loss.item())
                val_acc.append(acc)

            tbar.set_description('loss %s' % round(mean(val_loss), 10))
            tbar.update()                

        if i % 50 == 0:
            for param_group in optimizer.param_groups:
                print('Current lr: %s' % param_group['lr'])
            print(i, loss.item(), acc)

    current_acc = ((mean(train_acc) + mean(val_acc)) / 2)
    if current_acc > best_acc:
      current_acc = best_acc 



      model_save_name = 'Apr12_ONE_VS_ALL_LABEL28_WITH_IA.pt'
      #                    ^^ please remember to modify the file name to be saved! Do not duplicate file names to avoid error 
      # remember to save it as a pt file !!! 



      path = F'/content/drive/Shared drives/HaiHua Garbage Classification /Justin pretrained networks/One-VS-All/{model_save_name}'
      torch.save(model, path)
      print('whole model saved')

    else:
        pass

    print('Epoch: %s\n'
          'LOSS:\tTrain: %s\tVal: %s\n'
          'ACC:\tTrain: %s\tVal: %s' % (epoch+1, mean(train_loss), mean(val_loss), mean(train_acc), mean(val_acc)))


loading model...


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0/313 [00:00<?, ?it/s]

training...
trains for  274 steps per epoch


loss 0.0102626067:   0%|          | 1/313 [00:15<1:19:13, 15.23s/it]

...

  0%|          | 0/313 [00:00<?, ?it/s]

whole model saved
Epoch: 1
LOSS:	Train: 0.0033404777967573195	Val: 0.0021876779559725514
ACC:	Train: 0.7481751824817519	Val: 0.8421052631578947



loss 0.0005342436:   0%|          | 1/313 [00:14<1:13:27, 14.13s/it]

Current lr: 0.0001
0 0.0005342435906641185 1



loss 0.0016497436:  13%|█▎        | 42/313 [08:07<52:07, 11.54s/it]